# Loading & using trained weights of ANN model for MNIST

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

/home/igor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Data preparation
- importing data
- scaling data
- train-test splitting

In [2]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
mnist=fetch_mldata('MNIST original')
stdsc=StandardScaler()
#X,y=pd.DataFrame(mnist['data']), pd.DataFrame(mnist['target'])
X_all,y_all=mnist['data'], mnist['target'].astype(int)
X_all=stdsc.fit_transform(X_all)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.20, random_state=42)

/home/igor/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


- function for selcting batches:

In [3]:
def select_batch(X,y,batchsize):
    random_indices=np.random.randint(X.shape[0],size=batchsize)
    return X[random_indices], y[random_indices]

### Defining the neural network for classification


In [4]:
tf.reset_default_graph()
n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_outputs=10

# PLACEHOLDERS FOR INPUT
X=tf.placeholder(dtype=tf.float32,shape=(None, n_inputs), name="X")
y=tf.placeholder(shape=(None), dtype=tf.int32, name="y")

# LAYERS OF THE NEURAL NETWORK
with tf.name_scope("DNN"):
    hidden1=tf.layers.dense(inputs=X, units=n_hidden1, activation=tf.nn.relu, name='hidden1')
    hidden2=tf.layers.dense(inputs=hidden1, units=n_hidden2, activation=tf.nn.relu, name='hidden2')
    logits=tf.layers.dense(inputs=hidden2, units=n_outputs, activation=None, name='logits')

# LOSS FUNCTION    
with tf.name_scope("loss"):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(xentropy, name="loss")

# TRAINING OPERATIONS
learning_rate=0.01
#learning_rate=0.05
with tf.name_scope("train"):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer=tf.train.AdagradOptimizer(learning_rate=learning_rate)
    training_op=optimizer.minimize(loss)

# MEASURE FOR EVALUATING PERFORMANCE
with tf.name_scope("eval"):
    correct=tf.nn.in_top_k(targets=y,predictions=logits,k=1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))
    
init=tf.global_variables_initializer()
saver=tf.train.Saver()

### Loading the trained weights

In [13]:
with tf.Session() as sess:
    saver.restore(sess, "./saved_models/dnn_mnist_model.ckpt")
    Z=logits.eval(feed_dict={X: X_test, y: y_test})
    y_pred=np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./saved_models/dnn_mnist_model.ckpt


Accuracy:

In [14]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred=y_pred, y_true=y_test)

0.9710714285714286